<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class02/class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd